<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/use_image_as_input/training_spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import numpy as np

with open("/content/drive/MyDrive/pickle/metadata_for_GRF.pickle", mode="rb") as f:
   metadata_for_GRF = pickle.load(f)
f.close()

In [82]:
#ラベルづけする

TRIAL_num = 75732
class_label = 2

# 5クラス分類(HC vs H vs K vs A vs C)
# label_five_class = []
# for i in range(TRIAL_num):
#   label_five_class.append(metadata_for_GRF[i, class_label])

# 4クラス分類(H vs K vs A vs C)
# label_four_class = []
# for i in range(TRIAL_num):
#   if metadata_for_GRF[i, class_label] == 1:
#     label_four_class.append(0)
#   elif metadata_for_GRF[i, class_label] == 2: 
#     label_four_class.append(1)
#   elif metadata_for_GRF[i, class_label] == 3: 
#     label_four_class.append(2)
#   elif metadata_for_GRF[i, class_label] == 4: 
#     label_four_class.append(3)

# 2クラス分類(HC vs GD)
# label_HC_vs_GD_class = []
# for i in range(TRIAL_num):
#   if metadata_for_GRF[i, class_label] > 0:
#     label_HC_vs_GD_class.append(1)
#   else:
#     label_HC_vs_GD_class.append(0)

# 2クラス分類(HC vs H)
# label_HC_vs_H_class = []
# for i in range(TRIAL_num):
#   if metadata_for_GRF[i, class_label] == 0:
#     label_HC_vs_H_class.append(0)
#   elif metadata_for_GRF[i, class_label] == 1: 
#     label_HC_vs_H_class.append(1)

# 2クラス分類(HC vs K)
# label_HC_vs_K_class = []
# for i in range(TRIAL_num):
#   if metadata_for_GRF[i, class_label] == 0:
#     label_HC_vs_K_class.append(0)
#   elif metadata_for_GRF[i, class_label] == 2: 
#     label_HC_vs_K_class.append(1)

# 2クラス分類(HC vs A)
# label_HC_vs_A_class = []
# for i in range(TRIAL_num):
#   if metadata_for_GRF[i, class_label] == 0:
#     label_HC_vs_A_class.append(0)
#   elif metadata_for_GRF[i, class_label] == 3: 
#     label_HC_vs_A_class.append(1)

# 2クラス分類(HC vs C)
# label_HC_vs_C_class = []
# for i in range(TRIAL_num):
#   if metadata_for_GRF[i, class_label] == 0:
#     label_HC_vs_C_class.append(0)
#   elif metadata_for_GRF[i, class_label] == 4: 
#     label_HC_vs_C_class.append(1)

# train_X = np.array(train_data)
# train_y = np.array(train_data_label)
# test_X = np.array(test_data)
# test_y = np.array(test_data_label)

In [83]:
train_y = np.array(label_five_class)

In [84]:
with open("/content/drive/MyDrive/pickle/label_five_class.pickle", mode="wb") as f:
   pickle.dump(train_y, f)
f.close()

In [85]:
with open("/content/drive/MyDrive/pickle/label_five_class.pickle", mode="rb") as f:
   train_y = pickle.load(f)
f.close()

In [86]:
# データセット内訳チェック
print(np.array(np.where(train_y == 0)).shape)
print(np.array(np.where(train_y == 1)).shape)
print(np.array(np.where(train_y == 2)).shape)
print(np.array(np.where(train_y == 3)).shape)
print(np.array(np.where(train_y == 4)).shape)

# NaNチェック
print(np.isnan(train_y).sum())

(1, 7755)
(1, 12748)
(1, 19873)
(1, 21386)
(1, 13970)
0


In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers, regularizers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, InputLayer, Dense, Dropout, Activation, Flatten, concatenate, Conv1D, MaxPooling1D, Input, Reshape, Bidirectional
from keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
import time
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import seaborn as sns

# NAME = "HC-vs-C-LSTM_2class"
# NAME = "HC-vs-A-LSTM_2class"
# NAME = "HC-vs-K-LSTM_2class"
# NAME = "HC-vs-H-LSTM_2class"
# NAME = "HC-vs-GDs-LSTM_2class"
# NAME = "H-vs-K-vs-A-vs-C-LSTM_4class"
#NAME = "HC-vs-H-vs-K-vs-A-vs-C-LSTM_5class"
NAME = "H-vs-K-vs-A-vs-C-CNN_4class"
# NAME = "HC-vs-Patients-CNN_5class"
# NAME = "HC-vs-GDs-CNN_2class"
# NAME = "HC-vs-H-CNN_2class"
# NAME = "HC-vs-K-CNN_2class"
# NAME = "HC-vs-A-CNN_2class"
# NAME = "HC-vs-C-CNN_2class"

# 学習
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=10,
                )

# checkpointの設定
checkpoint = ModelCheckpoint(
                    filepath="saved_model/{}".format(NAME),
                    monitor='val_loss',
                    save_best_only=True,
                    period=1,
                )
initializer = tf.initializers.he_normal()
      
# CNN-A---------------------------------------------------------------------------------------------------------------------
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(6, 6), 
#                  kernel_initializer=initializer, 
#                  strides=(1, 1), 
#                  input_shape=(6, 101, 1)))
# model.add(Activation('relu'))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(4)) # 4クラス分類
# model.add(Activation('softmax'))

#-----------------------------------------------------------------------------------------------------------------------------

# CNN-A3---------------------------------------------------------------------------------------------------------------------
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(6, 3), 
#                  kernel_initializer=initializer, 
#                  strides=(1, 1), 
#                  input_shape=(6, 101, 1)))
# model.add(Activation('relu'))

# model.add(Conv2D(32, kernel_size=(1, 3), 
#                  strides=(1, 1)))
# model.add(Activation('relu'))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(4)) # 4クラス分類
# model.add(Activation('softmax'))

#-----------------------------------------------------------------------------------------------------------------------------

# CNN-A6---------------------------------------------------------------------------------------------------------------------
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(6, 6), 
#                  kernel_initializer=initializer, 
#                  strides=(1, 1), 
#                  input_shape=(6, 101, 1)))
# model.add(Activation('relu'))

# model.add(Conv2D(32, kernel_size=(1, 6), 
#                  strides=(1, 1)))
# model.add(Activation('relu'))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(4)) # 4クラス分類
# model.add(Activation('softmax'))

#-----------------------------------------------------------------------------------------------------------------------------

# CNN-C6---------------------------------------------------------------------------------------------------------------------
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(6, 6), 
#                  kernel_initializer=initializer, 
#                  strides=(1, 1), 
#                  input_shape=(6, 101, 1)))
# model.add(Activation('relu'))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(4)) # 4クラス分類
# model.add(Activation('softmax'))

#-----------------------------------------------------------------------------------------------------------------------------

# CNN-C3---------------------------------------------------------------------------------------------------------------------
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(3, 3), 
#                  kernel_initializer=initializer,
#                  strides=(1, 1), 
#                  input_shape=(10, 101, 1)))
# model.add(Activation('relu'))

# model.add(Conv2D(32, kernel_size=(3, 3), 
#                  strides=(1, 1)))
# model.add(Activation('relu'))

# model.add(Conv2D(32, kernel_size=(2, 2), 
#                  strides=(1, 1)))
# model.add(Activation('relu'))

# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(4)) # 4クラス分類
# model.add(Activation('softmax'))

#-----------------------------------------------------------------------------------------------------------------------------

# separated_1D-CNN----------------------------------------------------------------------------------------------------------------------
# dropout_rate = 0.5

# ML_affected_input = Input(shape=(101, 1), name="ML_affected")
# ML_unaffected_input = Input(shape=(101, 1), name="ML_unaffected")
# AP_affected_input = Input(shape=(101, 1), name="AP_affected")
# AP_unaffected_input = Input(shape=(101, 1), name="AP_unaffected")
# V_affected_input = Input(shape=(101, 1), name="V_affected")
# V_unaffected_input = Input(shape=(101, 1), name="V_unaffected")

# COP_ML_affected_input = Input(shape=(101, 1), name="COP_ML_affected")
# COP_ML_unaffected_input = Input(shape=(101, 1), name="COP_ML_unaffected")
# COP_AP_affected_input = Input(shape=(101, 1), name="COP_AP_affected")
# COP_AP_unaffected_input = Input(shape=(101, 1), name="COP_AP_unaffected")

# ML_affected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_affected_input)
# ML_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_affected_features)
# ML_affected_features = MaxPooling1D(pool_size=2)(ML_affected_features)
# ML_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_affected_features)
# ML_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_affected_features)
# ML_affected_features = MaxPooling1D(pool_size=2)(ML_affected_features)
# ML_affected_features = Flatten()(ML_affected_features)
# ML_affected_features = Dropout(rate=dropout_rate)(ML_affected_features)
# ML_affected_features = Dense(100, activation='relu')(ML_affected_features)

# ML_unaffected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_unaffected_input)
# ML_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_unaffected_features)
# ML_unaffected_features = MaxPooling1D(pool_size=2)(ML_unaffected_features)
# ML_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_unaffected_features)
# ML_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_unaffected_features)
# ML_unaffected_features = MaxPooling1D(pool_size=2)(ML_unaffected_features)
# ML_unaffected_features = Flatten()(ML_unaffected_features)
# ML_unaffected_features = Dropout(rate=dropout_rate)(ML_unaffected_features)
# ML_unaffected_features = Dense(100, activation='relu')(ML_unaffected_features)

# AP_affected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_affected_input)
# AP_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_affected_features)
# AP_affected_features = MaxPooling1D(pool_size=2)(AP_affected_features)
# AP_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_affected_features)
# AP_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_affected_features)
# AP_affected_features = MaxPooling1D(pool_size=2)(AP_affected_features)
# AP_affected_features = Flatten()(AP_affected_features)
# AP_affected_features = Dropout(rate=dropout_rate)(AP_affected_features)
# AP_affected_features = Dense(100, activation='relu')(AP_affected_features)

# AP_unaffected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_unaffected_input)
# AP_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_unaffected_features)
# AP_unaffected_features = MaxPooling1D(pool_size=2)(AP_unaffected_features)
# AP_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_unaffected_features)
# AP_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_unaffected_features)
# AP_unaffected_features = MaxPooling1D(pool_size=2)(AP_unaffected_features)
# AP_unaffected_features = Flatten()(AP_unaffected_features)
# AP_unaffected_features = Dropout(rate=dropout_rate)(AP_unaffected_features)
# AP_unaffected_features = Dense(100, activation='relu')(AP_unaffected_features)

# V_affected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_affected_input)
# V_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_affected_features)
# V_affected_features = MaxPooling1D(pool_size=2)(V_affected_features)
# V_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_affected_features)
# V_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_affected_features)
# V_affected_features = MaxPooling1D(pool_size=2)(V_affected_features)
# V_affected_features = Flatten()(V_affected_features)
# V_affected_features = Dropout(rate=dropout_rate)(V_affected_features)
# V_affected_features = Dense(100, activation='relu')(V_affected_features)

# V_unaffected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_unaffected_input)
# V_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_unaffected_features)
# V_unaffected_features = MaxPooling1D(pool_size=2)(V_unaffected_features)
# V_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_unaffected_features)
# V_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_unaffected_features)
# V_unaffected_features = MaxPooling1D(pool_size=2)(V_unaffected_features)
# V_unaffected_features = Flatten()(V_unaffected_features)
# V_unaffected_features = Dropout(rate=dropout_rate)(V_unaffected_features)
# V_unaffected_features = Dense(100, activation='relu')(V_unaffected_features)

# COP_ML_affected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_affected_input)
# COP_ML_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_affected_features)
# COP_ML_affected_features = MaxPooling1D(pool_size=2)(COP_ML_affected_features)
# COP_ML_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_affected_features)
# COP_ML_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_affected_features)
# COP_ML_affected_features = MaxPooling1D(pool_size=2)(COP_ML_affected_features)
# COP_ML_affected_features = Flatten()(COP_ML_affected_features)
# COP_ML_affected_features = Dropout(rate=dropout_rate)(COP_ML_affected_features)
# COP_ML_affected_features = Dense(100, activation='relu')(COP_ML_affected_features)

# COP_ML_unaffected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_unaffected_input)
# COP_ML_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_unaffected_features)
# COP_ML_unaffected_features = MaxPooling1D(pool_size=2)(COP_ML_unaffected_features)
# COP_ML_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_unaffected_features)
# COP_ML_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_ML_unaffected_features)
# COP_ML_unaffected_features = MaxPooling1D(pool_size=2)(COP_ML_unaffected_features)
# COP_ML_unaffected_features = Flatten()(COP_ML_unaffected_features)
# COP_ML_unaffected_features = Dropout(rate=dropout_rate)(COP_ML_unaffected_features)
# COP_ML_unaffected_features = Dense(100, activation='relu')(COP_ML_unaffected_features)

# COP_AP_affected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_affected_input)
# COP_AP_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_affected_features)
# COP_AP_affected_features = MaxPooling1D(pool_size=2)(COP_AP_affected_features)
# COP_AP_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_affected_features)
# COP_AP_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_affected_features)
# COP_AP_affected_features = MaxPooling1D(pool_size=2)(COP_AP_affected_features)
# COP_AP_affected_features = Flatten()(COP_AP_affected_features)
# COP_AP_affected_features = Dropout(rate=dropout_rate)(COP_AP_affected_features)
# COP_AP_affected_features = Dense(100, activation='relu')(COP_AP_affected_features)

# COP_AP_unaffected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_unaffected_input)
# COP_AP_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_unaffected_features)
# COP_AP_unaffected_features = MaxPooling1D(pool_size=2)(COP_AP_unaffected_features)
# COP_AP_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_unaffected_features)
# COP_AP_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(COP_AP_unaffected_features)
# COP_AP_unaffected_features = MaxPooling1D(pool_size=2)(COP_AP_unaffected_features)
# COP_AP_unaffected_features = Flatten()(COP_AP_unaffected_features)
# COP_AP_unaffected_features = Dropout(rate=dropout_rate)(COP_AP_unaffected_features)
# COP_AP_unaffected_features = Dense(100, activation='relu')(COP_AP_unaffected_features)

# concat = concatenate([ML_affected_features, ML_unaffected_features, AP_affected_features, AP_unaffected_features, V_affected_features, V_unaffected_features, COP_ML_affected_features, COP_ML_unaffected_features, COP_AP_affected_features, COP_AP_unaffected_features])
# # concat = concatenate([V_affected_features, V_unaffected_features])
# concat = Dropout(rate=dropout_rate)(concat)
# Dense1 = Dense(100, activation='relu')(concat)
# Dense1 = Dropout(rate=dropout_rate)(Dense1)
# Dense2 = Dense(20, activation='relu')(Dense1)
# output = Dense(4, activation='softmax')(Dense2)

# model = Model(
#     inputs=[ML_affected_input, ML_unaffected_input, AP_affected_input, AP_unaffected_input, V_affected_input, V_unaffected_input, COP_ML_affected_input, COP_ML_unaffected_input, COP_AP_affected_input, COP_AP_unaffected_input],
#     # inputs=[V_affected_input, V_unaffected_input],
#     outputs=output,
# )
#-----------------------------------------------------------------------------------------------------------------------------


#同じ座標のデータ同士まとめて特徴抽出 * 1DCNN-------------------------------------------------------------------------------------------------------------------------
# dropout_rate = 0.5

# ML_input = Input(shape=(202, 1), name="ML")
# AP_input = Input(shape=(202, 1), name="AP")
# V_input = Input(shape=(202, 1), name="V")

# ML_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_input)
# ML_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_features)
# ML_features = MaxPooling1D(pool_size=2)(ML_features)
# ML_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_features)
# ML_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(ML_features)
# ML_features = MaxPooling1D(pool_size=2)(ML_features)
# ML_features = Flatten()(ML_features)
# ML_features = Dropout(rate=dropout_rate)(ML_features)
# ML_features = Dense(100, activation='relu')(ML_features)

# AP_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_input)
# AP_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_features)
# AP_features = MaxPooling1D(pool_size=2)(AP_features)
# AP_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_features)
# AP_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(AP_features)
# AP_features = MaxPooling1D(pool_size=2)(AP_features)
# AP_features = Flatten()(AP_features)
# AP_features = Dropout(rate=dropout_rate)(AP_features)
# AP_features = Dense(100, activation='relu')(AP_features)

# V_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_input)
# V_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_features)
# V_features = MaxPooling1D(pool_size=2)(V_features)
# V_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_features)
# V_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(V_features)
# V_features = MaxPooling1D(pool_size=2)(V_features)
# V_features = Flatten()(V_features)
# V_features = Dropout(rate=dropout_rate)(V_features)
# V_features = Dense(100, activation='relu')(V_features)

# concat = concatenate([ML_features, AP_features, V_features])
# concat = Dropout(rate=dropout_rate)(concat)
# Dense1 = Dense(100, activation='relu')(concat)
# Dense1 = Dropout(rate=dropout_rate)(Dense1)
# Dense2 = Dense(20, activation='relu')(Dense1)
# output = Dense(4, activation='softmax')(Dense2)

# model = Model(
#     inputs=[ML_input, AP_input, V_input],
#     # inputs=[V_affected_input, V_unaffected_input],
#     outputs=output,
# )
#-----------------------------------------------------------------------------------------------------------------------------

#同じ座標のデータ同士まとめて特徴抽出 * 2DCNN-------------------------------------------------------------------------------------------------------------------------
dropout_rate = 0.5
from keras import backend
backend.set_image_data_format('channels_last')

ML_input = Input(shape=(101, 2, 1), name="ML")
AP_input = Input(shape=(101, 2, 1), name="AP")
V_input = Input(shape=(101, 2, 1), name="V")

ML_features = Conv2D(64, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(ML_input)
ML_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(ML_features)
ML_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(ML_features)
ML_features = Flatten()(ML_features)
ML_features = Dense(256, activation='relu')(ML_features)

AP_features = Conv2D(64, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(AP_input)
AP_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(AP_features)
AP_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(AP_features)
AP_features = Flatten()(AP_features)
AP_features = Dense(256, activation='relu')(AP_features)

V_features = Conv2D(64, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(V_input)
V_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(V_features)
V_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, padding='same')(V_features)
V_features = Flatten()(V_features)
V_features = Dense(256, activation='relu')(V_features)

concat = concatenate([ML_features, AP_features, V_features])
concat = Dropout(rate=dropout_rate)(concat)
# Dense1 = Dense(100, activation='relu')(concat)
# Dense1 = Dropout(rate=dropout_rate)(Dense1)
# Dense2 = Dense(20, activation='relu')(Dense1)
output = Dense(4, activation='softmax')(concat)

model = Model(
    inputs=[ML_input, AP_input, V_input],
    # inputs=[V_affected_input, V_unaffected_input],
    outputs=output,
)
#-----------------------------------------------------------------------------------------------------------------------------

#AffectedとUnaffectedをわける * 1DCNN-----------------------------------------------------------------------------------------------------------------------
# dropout_rate = 0.5

# GRF_affected_input = Input(shape=(303, 1), name="GRF_affected")
# GRF_unaffected_input = Input(shape=(303, 1), name="GRF_unaffected")

# GRF_affected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_affected_input)
# GRF_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_affected_features)
# GRF_affected_features = MaxPooling1D(pool_size=2)(GRF_affected_features)
# GRF_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_affected_features)
# GRF_affected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_affected_features)
# GRF_affected_features = MaxPooling1D(pool_size=2)(GRF_affected_features)
# GRF_affected_features = Flatten()(GRF_affected_features)
# GRF_affected_features = Dropout(rate=dropout_rate)(GRF_affected_features)
# GRF_affected_features = Dense(100, activation='relu')(GRF_affected_features)

# GRF_unaffected_features = Conv1D(8, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_unaffected_input)
# GRF_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_unaffected_features)
# GRF_unaffected_features = MaxPooling1D(pool_size=2)(GRF_unaffected_features)
# GRF_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_unaffected_features)
# GRF_unaffected_features = Conv1D(16, activation='relu', kernel_size=3, kernel_initializer=initializer, strides=2)(GRF_unaffected_features)
# GRF_unaffected_features = MaxPooling1D(pool_size=2)(GRF_unaffected_features)
# GRF_unaffected_features = Flatten()(GRF_unaffected_features)
# GRF_unaffected_features = Dropout(rate=dropout_rate)(GRF_unaffected_features)
# GRF_unaffected_features = Dense(100, activation='relu')(GRF_unaffected_features)

# concat = concatenate([GRF_affected_features, GRF_unaffected_features])
# concat = Dropout(rate=dropout_rate)(concat)
# Dense1 = Dense(100, activation='relu')(concat)
# Dense1 = Dropout(rate=dropout_rate)(Dense1)
# Dense2 = Dense(20, activation='relu')(Dense1)
# output = Dense(4, activation='softmax')(Dense2)

# model = Model(
#     inputs=[GRF_affected_input, GRF_unaffected_input],
#     outputs=output,
# )

#AffectedとUnaffectedをわける * 2DCNN------------------------------------------------------------------------------------------------------------------
# from keras import backend
# backend.set_image_data_format('channels_last')

# dropout_rate = 0.5

# GRF_affected_input = Input(shape=(101, 3, 1), name="GRF_affected")
# GRF_unaffected_input = Input(shape=(101, 3, 1), name="GRF_unaffected")

# GRF_affected_features = Conv2D(64, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, strides=(1, 1), padding='same')(GRF_affected_input)
# # GRF_affected_features = MaxPooling2D(pool_size=2)(GRF_affected_features)
# GRF_affected_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, strides=(1, 1), padding='same')(GRF_affected_features)
# # GRF_affected_features = MaxPooling2D(pool_size=2, padding='same')(GRF_affected_features)
# GRF_affected_features = Flatten()(GRF_affected_features)
# # GRF_affected_features = Dropout(rate=dropout_rate)(GRF_affected_features)
# GRF_affected_features = Dense(256, activation='relu')(GRF_affected_features)

# GRF_unaffected_features = Conv2D(64, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, strides=(1, 1), padding='same')(GRF_unaffected_input)
# # GRF_unaffected_features = MaxPooling2D(pool_size=2)(GRF_unaffected_features)
# GRF_unaffected_features = Conv2D(128, activation='relu', kernel_size=(3, 3), kernel_initializer=initializer, strides=(1, 1), padding='same')(GRF_unaffected_features)
# # GRF_unaffected_features = MaxPooling2D(pool_size=2, padding='same')(GRF_unaffected_features)
# GRF_unaffected_features = Flatten()(GRF_unaffected_features)
# # GRF_unaffected_features = Dropout(rate=dropout_rate)(GRF_unaffected_features)
# GRF_unaffected_features = Dense(256, activation='relu')(GRF_unaffected_features)

# concat = concatenate([GRF_affected_features, GRF_unaffected_features])
# concat = Dropout(rate=dropout_rate)(concat)
# # Dense1 = Dense(100, activation='relu')(concat)
# # Dense1 = Dropout(rate=dropout_rate)(Dense1)
# # Dense2 = Dense(20, activation='relu')(Dense1)
# output = Dense(4, activation='softmax')(concat)

# model = Model(
#     inputs=[GRF_affected_input, GRF_unaffected_input],
#     outputs=output,
# )
#-----------------------------------------------------------------------------------------------------------------------------

# 共通

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# 5クラス分類, 4クラス分類
# model.compile(loss='sparse_categorical_crossentropy', 
#               optimizer= optimizers.Adam(lr=0.002, beta_1= 0.9, beta_2= 0.999),
#               metrics=['accuracy'],
#               )

model.compile(loss='sparse_categorical_crossentropy',
              optimizer= optimizers.SGD(lr=5e-2, decay=1e-2),
              metrics=['accuracy'],
              )

# 2クラス分類
# model.compile(loss='binary_crossentropy',
#               optimizer= optimizers.SGD(lr=5e-3),
#               metrics=['accuracy'],
#               )

model.summary()

# X = np.reshape(train_X, (-1, 101, 10))
# reshaped_ML_affected = np.reshape(X[::, ::, 0], (X.shape[0], X.shape[1], 1))
# reshaped_ML_unaffected = np.reshape(X[::, ::, 5], (X.shape[0], X.shape[1], 1))
# reshaped_AP_affected = np.reshape(X[::, ::, 1], (X.shape[0], X.shape[1], 1))
# reshaped_AP_unaffected = np.reshape(X[::, ::, 6], (X.shape[0], X.shape[1], 1))
# reshaped_V_affected = np.reshape(X[::, ::, 2], (X.shape[0], X.shape[1], 1))
# reshaped_V_unaffected = np.reshape(X[::, ::, 7], (X.shape[0], X.shape[1], 1))
# reshaped_COP_ML_affected = np.reshape(X[::, ::, 3], (X.shape[0], X.shape[1], 1))
# reshaped_COP_ML_unaffected = np.reshape(X[::, ::, 8], (X.shape[0], X.shape[1], 1))
# reshaped_COP_AP_affected = np.reshape(X[::, ::, 4], (X.shape[0], X.shape[1], 1))
# reshaped_COP_AP_unaffected = np.reshape(X[::, ::, 9], (X.shape[0], X.shape[1], 1))

X = np.reshape(train_X, (-1, 101, 6))
reshaped_ML_affected = np.reshape(X[::, ::, 0], (X.shape[0], X.shape[1], 1))
reshaped_ML_unaffected = np.reshape(X[::, ::, 3], (X.shape[0], X.shape[1], 1))
reshaped_AP_affected = np.reshape(X[::, ::, 1], (X.shape[0], X.shape[1], 1))
reshaped_AP_unaffected = np.reshape(X[::, ::, 4], (X.shape[0], X.shape[1], 1))
reshaped_V_affected = np.reshape(X[::, ::, 2], (X.shape[0], X.shape[1], 1))
reshaped_V_unaffected = np.reshape(X[::, ::, 5], (X.shape[0], X.shape[1], 1))

reshaped_ML = np.concatenate([reshaped_ML_affected, reshaped_ML_unaffected], axis=2)
reshaped_AP = np.concatenate([reshaped_AP_affected, reshaped_AP_unaffected], axis=2)
reshaped_V = np.concatenate([reshaped_V_affected, reshaped_V_unaffected], axis=2)

reshaped_ML = np.reshape(reshaped_ML, (-1, reshaped_ML.shape[1], reshaped_ML.shape[2], 1))
reshaped_AP = np.reshape(reshaped_AP, (-1, reshaped_AP.shape[1], reshaped_AP.shape[2], 1))
reshaped_V = np.reshape(reshaped_V, (-1, reshaped_V.shape[1], reshaped_V.shape[2], 1))

# reshaped_GRF_affected = np.concatenate([reshaped_ML_affected, reshaped_AP_affected, reshaped_V_affected], axis=2)
# reshaped_GRF_unaffected = np.concatenate([reshaped_ML_unaffected, reshaped_AP_unaffected, reshaped_V_unaffected], axis=2)

# reshaped_GRF_affected = np.reshape(reshaped_GRF_affected, (-1, reshaped_GRF_affected.shape[1], reshaped_GRF_affected.shape[2], 1))
# reshaped_GRF_unaffected = np.reshape(reshaped_GRF_affected, (-1, reshaped_GRF_unaffected.shape[1], reshaped_GRF_unaffected.shape[2], 1))

# model.fit({"GRF_affected": reshaped_GRF_affected,
#            "GRF_unaffected": reshaped_GRF_unaffected, 
#           #  "COP_AP_affected": reshaped_COP_AP_affected, "COP_AP_unaffected": reshaped_COP_AP_unaffected,
#           #  "COP_ML_affected": reshaped_COP_ML_affected, "COP_ML_unaffected": reshaped_COP_ML_unaffected,
#            }, 
#           train_y,
#           batch_size=10,
#           epochs=1000,
#           validation_split=0.4,
#           callbacks=[tensorboard, checkpoint, early_stopping])

model.fit({"ML": reshaped_ML,
           "AP": reshaped_AP, 
           "V": reshaped_V, 
          #  "COP_AP_affected": reshaped_COP_AP_affected, "COP_AP_unaffected": reshaped_COP_AP_unaffected,
          #  "COP_ML_affected": reshaped_COP_ML_affected, "COP_ML_unaffected": reshaped_COP_ML_unaffected,
           }, 
          train_y,
          batch_size=10,
          epochs=100,
          validation_split=0.4,
          callbacks=[tensorboard, checkpoint, early_stopping])

# model.fit({"ML_affected": reshaped_ML_affected, "ML_unaffected": reshaped_ML_unaffected,
#            "AP_affected": reshaped_AP_affected, "AP_unaffected": reshaped_AP_unaffected,
#            "V_affected": reshaped_V_affected, "V_unaffected": reshaped_V_unaffected,
#           #  "COP_AP_affected": reshaped_COP_AP_affected, "COP_AP_unaffected": reshaped_COP_AP_unaffected,
#           #  "COP_ML_affected": reshaped_COP_ML_affected, "COP_ML_unaffected": reshaped_COP_ML_unaffected,
#            }, 
#           train_y,
#           batch_size=10,
#           epochs=100,
#           validation_split=0.2,
#           callbacks=[tensorboard, checkpoint])

# model.fit({
#            "V_affected": reshaped_V_affected, "V_unaffected": reshaped_V_unaffected,
#            }, 
#           train_y,
#           batch_size=10,
#           epochs=100,
#           validation_split=0.2,
#           callbacks=[tensorboard, checkpoint])

# X = np.reshape(train_X, (-1, 10, 101, 1))
# model.fit(X, 
#           train_y,
#           batch_size=3,
#           epochs=1000,
#           validation_split=0.2,
#           shuffle=True,
#           callbacks=[tensorboard, checkpoint])

# training parameters = kernel_size * (1Dだったらここは1, 2Dだったらここはkernel_size) * output_channels + num_bias
# 216 = 8 * 1 * 24 + 24

In [ ]:
!unzip /content/drive/MyDrive/AP_left.zip
!unzip /content/drive/MyDrive/AP_right.zip
!unzip /content/drive/MyDrive/AP_right_69826.zip
!unzip /content/drive/MyDrive/ML_left.zip
!unzip /content/drive/MyDrive/ML_left_58106.zip
!unzip /content/drive/MyDrive/ML_right.zip
!unzip /content/drive/MyDrive/V_left.zip
!unzip /content/drive/MyDrive/V_left_48327.zip
!unzip /content/drive/MyDrive/V_right.zip
!unzip /content/drive/MyDrive/V_right_34533.zip